In [1]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.2 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import tensorflow as tf
from tabulate import tabulate
from tqdm import trange
import random
import tqdm
from tensorflow.keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt

In [3]:
torch.cuda.is_available()

True

In [4]:
df = pd.read_csv("./augmented_df.csv")
# df = df.rename(columns={"Unnamed: 0": "label_id"})

In [5]:
df.head()

,category,text
0,African daisy Alternaria alternata,Alternaria alternata has many different hosts ...
1,African daisy Alternaria alternata,"Alternaria alternata, or AAL, is a fungal path..."
2,African daisy Alternaria dauci,Alternaria Leaf Blight is a foliar disease of ...
3,African daisy Alternaria dauci,Alternaria Leaf Blight is a foliar disease of ...
4,African daisy Aphelenchoides fragariae,"In fields, symptomatic plants are recognized i..."


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [7]:
text = df.text.values
labels = df.category.values

In [8]:
len(labels)

3914

In [9]:
label_to_id = {}
id_to_label = {}
j = 0
for i in np.unique(labels):
  label_to_id[i] = j
  id_to_label[j] = i
  j += 1

In [10]:
label_id = []
for i in labels:
  label_id.append(label_to_id[i])

In [11]:
len(label_to_id.keys())

1913

In [12]:
df['label_id']=label_id

In [13]:
df=df.drop(columns=['category'])

In [14]:
df.head()

,text,label_id
0,Alternaria alternata has many different hosts ...,0
1,"Alternaria alternata, or AAL, is a fungal path...",0
2,Alternaria Leaf Blight is a foliar disease of ...,1
3,Alternaria Leaf Blight is a foliar disease of ...,1
4,"In fields, symptomatic plants are recognized i...",2


In [ ]:
history = model.fit([trid,tram],labels2_train,validation_data=([teid,team],labels2_test), epochs=32)

Epoch 1/32
56/56 [==============================] - 148s 2s/step - loss: 4.5220 - accuracy: 0.0261 - val_loss: 4.4906 - val_accuracy: 0.0306
Epoch 2/32
56/56 [==============================] - 96s 2s/step - loss: 4.4873 - accuracy: 0.0272 - val_loss: 4.5219 - val_accuracy: 0.0255
Epoch 3/32
56/56 [==============================] - 96s 2s/step - loss: 4.4541 - accuracy: 0.0312 - val_loss: 4.4626 - val_accuracy: 0.0408
Epoch 4/32
56/56 [==============================] - 96s 2s/step - loss: 4.4342 - accuracy: 0.0385 - val_loss: 4.4984 - val_accuracy: 0.0408
Epoch 5/32
56/56 [==============================] - 96s 2s/step - loss: 4.4273 - accuracy: 0.0385 - val_loss: 4.4838 - val_accuracy: 0.0408
Epoch 6/32
56/56 [==============================] - 96s 2s/step - loss: 4.4242 - accuracy: 0.0385 - val_loss: 4.4620 - val_accuracy: 0.0408
Epoch 7/32
56/56 [==============================] - 96s 2s/step - loss: 4.4185 - accuracy: 0.0385 - val_loss: 4.5034 - val_accuracy: 0.0408
Epoch 8/32
56/56 [=

KeyboardInterrupt: ignored

In [15]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [16]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in enumerate(df['text']):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [17]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

In [18]:
X_input_ids.shape

(3914, 256)

In [19]:
labels2 = np.zeros((len(df), len(set(label_id))))

In [20]:
labels2[np.arange(len(df)), label_id] = 1

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels2))
dataset.take(196)

<_TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(1913,), dtype=tf.float64, name=None))>

In [22]:
def TCDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [23]:
dataset = dataset.map(TCDatasetMapFunction)

In [24]:
dataset.take(196)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(1913,), dtype=tf.float64, name=None))>

In [25]:
dataset = dataset.shuffle(2000).batch(32, drop_remainder=True)

In [26]:
p = 0.8
train_size = int((len(df)//16)*p)

In [27]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [28]:
from transformers import TFBertModel

In [29]:
model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [42]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(1024, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

tc_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [44]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
tc_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = tc_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=16
)

Epoch 1/16
122/122 [==============================] - 238s 2s/step - loss: 7.5874 - accuracy: 5.1230e-04
Epoch 2/16
122/122 [==============================] - 197s 2s/step - loss: 7.5658 - accuracy: 0.0018
Epoch 3/16
122/122 [==============================] - 197s 2s/step - loss: 7.5572 - accuracy: 0.0026
Epoch 4/16
122/122 [==============================] - 197s 2s/step - loss: 7.5371 - accuracy: 0.0046
Epoch 5/16
122/122 [==============================] - 197s 2s/step - loss: 7.4864 - accuracy: 0.0059
Epoch 6/16
122/122 [==============================] - 196s 2s/step - loss: 7.3795 - accuracy: 0.0090
Epoch 7/16
122/122 [==============================] - 196s 2s/step - loss: 7.2262 - accuracy: 0.0126
Epoch 8/16
122/122 [==============================] - 197s 2s/step - loss: 7.0556 - accuracy: 0.0149
Epoch 9/16
122/122 [==============================] - 197s 2s/step - loss: 6.8770 - accuracy: 0.0172
Epoch 10/16
122/122 [==============================] - 197s 2s/step - loss: 6.6973 - ac

In [30]:
input_ids.shape

TensorShape([None, 256])

In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
intermediate_layer2 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer2')(intermediate_layer)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer2) # softmax -> calcs probs of classes

tc_model2 = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model2.summary()
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
tc_model2.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = tc_model2.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=32
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [30]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(1024, activation='relu', name='intermediate_layer')(bert_embds)
intermediate_layer2 = tf.keras.layers.Dense(256, activation='relu', name='intermediate_layer2')(intermediate_layer)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer2) # softmax -> calcs probs of classes

tc_model2 = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model2.summary()
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
tc_model2.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = tc_model2.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=64
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

tc_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
tc_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
hist = tc_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=32
)

Epoch 1/32
97/97 [==============================] - 145s 1s/step - loss: 4.6952 - accuracy: 0.0129 - val_loss: 4.5530 - val_accuracy: 0.0350
Epoch 2/32
97/97 [==============================] - 95s 981ms/step - loss: 4.5254 - accuracy: 0.0387 - val_loss: 4.4934 - val_accuracy: 0.0250
Epoch 3/32
97/97 [==============================] - 95s 975ms/step - loss: 4.4717 - accuracy: 0.0387 - val_loss: 4.4382 - val_accuracy: 0.0350
Epoch 4/32
97/97 [==============================] - 97s 997ms/step - loss: 4.4529 - accuracy: 0.0322 - val_loss: 4.4295 - val_accuracy: 0.0400
Epoch 5/32
97/97 [==============================] - 95s 978ms/step - loss: 4.4349 - accuracy: 0.0374 - val_loss: 4.4393 - val_accuracy: 0.0300
Epoch 6/32
97/97 [==============================] - 95s 980ms/step - loss: 4.3945 - accuracy: 0.0451 - val_loss: 4.3518 - val_accuracy: 0.0550
Epoch 7/32
97/97 [==============================] - 94s 974ms/step - loss: 4.3982 - accuracy: 0.0522 - val_loss: 4.3240 - val_accuracy: 0.0750
E

In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
intermediate_layer2 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer2')(intermediate_layer)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer2) # softmax -> calcs probs of classes

tc_model2 = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model2.summary()
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
tc_model2.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = tc_model2.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=32
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
intermediate_layer2 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer2')(intermediate_layer)
intermediate_layer3 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer3')(intermediate_layer2)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer3) # softmax -> calcs probs of classes

tc_model3 = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model3.summary()
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
tc_model3.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = tc_model3.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=32
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
large_bert = TFBertModel.from_pretrained('bert-large-cased')

Some layers from the model checkpoint at bert-large-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = large_bert.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

tc_model4 = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model4.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  333579264   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
tc_model4.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = tc_model4.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=8
)